In [2]:
import polars as pl
import pandas as pd
import numpy as np
import pyarrow

import matplotlib.pyplot as plt
import seaborn as sns
# https://github.com/martinbel/polars-tutorial/blob/master/03-Polars.ipynb

In [5]:
df = pl.read_csv("2019-Oct-reduced.csv")
df.head()

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
str,str,i64,f64,str,str,f64,i64,str
"""2019-10-01 00:…","""view""",44600062,2.1038e18,null,"""shiseido""",35.79,541312140,"""72d76fde-8bb3-…"
"""2019-10-01 00:…","""view""",3900821,2.0530e18,"""appliances.env…","""aqua""",33.2,554748717,"""9333dfbd-b87a-…"
"""2019-10-01 00:…","""view""",17200506,2.0530e18,"""furniture.livi…",null,543.1,519107250,"""566511c2-e2e3-…"
"""2019-10-01 00:…","""view""",1307067,2.0530e18,"""computers.note…","""lenovo""",251.74,550050854,"""7c90fc70-0e80-…"
"""2019-10-01 00:…","""view""",1004237,2.0530e18,"""electronics.sm…","""apple""",1081.98,535871217,"""c6bd7419-2748-…"


# 1. Groupby - pl.count (one group)

In [12]:
# agg nikalne ke liye count kiya hai
df.groupby('brand').agg([
    pl.count()
]).head(10)

brand,count
str,u32
"""giti""",2
"""candide""",10
"""vissol""",3
"""jura""",24
"""bessey""",1
"""converse""",30
"""ariston""",2003
"""aiken""",8
"""hori""",3


In [20]:
df.groupby('brand').agg([
    pl.count()
]).sort(by='count',descending = True)

brand,count
str,u32
null,155594
"""samsung""",126027
"""apple""",107634
"""xiaomi""",72507
"""huawei""",29360
"""lucente""",17799
"""bosch""",12959
"""lg""",11495
"""acer""",10214


In [25]:
# aggregate in polars and convert to pandas for sorting the aggregation
df_count = df.groupby("brand").agg([
    pl.count()
]).to_pandas().sort_values("count", ascending=False)

print(df_count.shape)

df_count.head()

(2246, 2)


,brand,count
1674,None,155594
1919,samsung,126027
1795,apple,107634
343,xiaomi,72507
1305,huawei,29360


In [35]:
# Compute count and percent of instances
(
   df.groupby('price').agg([
      pl.count()
])
    .sort("count" , descending=True)
    .with_columns([
        (pl.col('count')/pl.col('count')).sum().alias('mean')
    ])
).head(8)

price,count,mean
f64,u32,f64
975.56,7997,17547.0
975.57,7324,17547.0
130.7,6833,17547.0
1415.48,6384,17547.0
257.15,6240,17547.0
161.93,4830,17547.0
231.64,4627,17547.0
203.35,4409,17547.0


In [36]:
df.sample(5)

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
str,str,i64,f64,str,str,f64,i64,str
"""2019-10-01 13:…","""view""",2800432,2.0530e18,"""appliances.kit…",null,200.26,516111798,"""bedffb97-9368-…"
"""2019-10-01 08:…","""view""",1004856,2.0530e18,"""electronics.sm…","""samsung""",130.7,537315270,"""20de1ee5-4496-…"
"""2019-10-01 03:…","""view""",1002995,2.0530e18,"""electronics.sm…","""apple""",456.79,518437642,"""dbacda92-2d53-…"
"""2019-10-01 15:…","""view""",3800545,2.0530e18,"""appliances.iro…","""braun""",190.46,515918427,"""69a8b639-1d1f-…"
"""2019-10-01 11:…","""view""",1005157,2.0530e18,"""electronics.sm…","""xiaomi""",282.89,512407705,"""37c11309-135b-…"


## Value Counts works as expected

In [37]:
# this works!
df['event_type'].value_counts()

# More verbose version with groupby
df.groupby("event_type").agg(pl.count())

event_type,count
str,u32
"""cart""",15040
"""purchase""",17296
"""view""",1016239


# 2. Groupby - pl.count (two groups)

In [40]:
df_brand_event = df.groupby('brand','event_type').agg([
    pl.count()
])
df_brand_event.head()

brand,event_type,count
str,str,u32
null,"""view""",154052
"""pulser""","""view""",2450
"""baden""","""view""",2738
"""huawei""","""view""",27856
"""elenberg""","""view""",6939


In [49]:
# Convert from long to wide format
df_brand_event_wide = df_brand_event.pivot(values=["count"]ZZ, 
                                           index=["brand"], 
                                           columns=['event_type'])
df_brand_event_wide.head()

C:\Users\Khuzaima\AppData\Local\Temp\ipykernel_4492\2015406592.py:2: DeprecationWarning: In a future version of polars, the default `aggregate_function` will change from `'first'` to `None`. Please pass `'first'` to keep the current behaviour, or `None` to accept the new one.
  df_brand_event_wide = df_brand_event.pivot(values=["count"],


brand,view,purchase,cart
str,u32,u32,u32
null,154052,1388,154
"""pulser""",2450,22,null
"""baden""",2738,11,null
"""huawei""",27856,617,887
"""elenberg""",6939,167,null


In [52]:
# Pivot and sorting
df_brand_event_wide = ( 
    df_brand_event
    .pivot(values="count", index=["brand"], columns=['event_type'])
    .sort("purchase", descending=True)
)

df_brand_event_wide.head()

C:\Users\Khuzaima\AppData\Local\Temp\ipykernel_4492\4258880573.py:3: DeprecationWarning: In a future version of polars, the default `aggregate_function` will change from `'first'` to `None`. Please pass `'first'` to keep the current behaviour, or `None` to accept the new one.
  df_brand_event


brand,view,purchase,cart
str,u32,u32,u32
"""samsung""",116573,4053,5401
"""apple""",99822,3565,4247
null,154052,1388,154
"""xiaomi""",68918,1337,2252
"""huawei""",27856,617,887


In [53]:
# Keep original columns and compute % values
agg_performance = df_brand_event_wide.with_columns([
    (pl.col("cart") / pl.col("view")).alias("cart_by_views"),
    (pl.col("purchase") / pl.col("cart")).alias("buy_by_cart"),
    (pl.col("purchase") / pl.col("view")).alias("buy_by_views"),
])
agg_performance.head()

brand,view,purchase,cart,cart_by_views,buy_by_cart,buy_by_views
str,u32,u32,u32,f64,f64,f64
"""samsung""",116573,4053,5401,0.046331,0.750417,0.034768
"""apple""",99822,3565,4247,0.042546,0.839416,0.035714
null,154052,1388,154,0.001,9.012987,0.00901
"""xiaomi""",68918,1337,2252,0.032677,0.593694,0.0194
"""huawei""",27856,617,887,0.031842,0.695603,0.02215


In [62]:
agg_performance.filter(pl.col('buy_by_cart')<1)\
.select(['buy_by_cart','buy_by_views','cart_by_views'])\
.to_pandas().quantile([0, 0.25, 0.5, 0.75, 0.99, 1])

# .boxplot()

,buy_by_cart,buy_by_views,cart_by_views
0.00,0.368421,0.002288,0.004577
0.25,0.500000,0.008299,0.013889
0.50,0.593694,0.011259,0.018860
0.75,0.750000,0.021958,0.032677
0.99,0.837956,0.062664,0.114017
1.00,0.839416,0.071174,0.124555


## 3. Group By - UserId + 2 Groups

In [75]:
# groupby 
# %%timeit
# 8.72 s ± 246 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

df_user_brand_event = df.groupby(["user_id", "brand", "event_type"]).agg([
    pl.count()
])

df_user_brand_event.head()

user_id,brand,event_type,count
i64,str,str,u32
541312140,"""shiseido""","""view""",2
520571932,"""baden""","""view""",2
555446365,"""brw""","""view""",2
513642368,null,"""view""",2
537192226,"""haier""","""view""",1


In [64]:
df_user_brand_event.shape

(396699, 4)

In [77]:
# pivot and sort
df_user_brand_event = (df_user_brand_event
    .pivot(values="count", index=["user_id", "brand"], columns=['event_type'])
    .sort("purchase", descending=True)
)
df_user_brand_event.shape

C:\Users\Khuzaima\AppData\Local\Temp\ipykernel_4492\3434727870.py:2: DeprecationWarning: In a future version of polars, the default `aggregate_function` will change from `'first'` to `None`. Please pass `'first'` to keep the current behaviour, or `None` to accept the new one.
  df_user_brand_event = (df_user_brand_event


(374482, 5)

In [79]:
df_user_brand_event.head()

user_id,brand,view,cart,purchase
i64,str,u32,u32,u32
554090147,"""apple""",37,24,26
548891541,"""ariston""",33,17,19
541337272,"""samsung""",34,24,17
520695559,"""apple""",23,7,16
531974857,"""samsung""",24,11,14


In [80]:
df_user_brand_event.sample(10)


user_id,brand,view,cart,purchase
i64,str,u32,u32,u32
554870604,"""fassen""",1,null,null
513194193,"""vitek""",4,null,null
512421019,"""xiaomi""",2,null,null
514168979,"""milwaukee""",1,null,null
516029862,"""samsung""",1,null,null
524566360,"""phoenix""",1,null,null
514303423,"""samsung""",2,null,null
517233496,"""apple""",4,null,null
552869893,"""samsung""",2,null,null


In [81]:
df_user_brand_event = df_user_brand_event.with_columns([
    pl.col('purchase').fill_null(strategy="zero"),
    pl.col('view').fill_null(strategy="zero"),
    pl.col('cart').fill_null(strategy="zero"),
])

In [83]:
df_user_brand_event.sample(10)

user_id,brand,view,cart,purchase
i64,str,u32,u32,u32
545567705,"""artmusical""",1,0,0
512931831,"""bosch""",23,0,0
535956210,"""samsung""",3,0,0
514587373,null,7,0,0
544712254,"""autoprofi""",1,0,0
552659763,"""xiaomi""",3,2,1
555538036,"""huawei""",2,0,0
513300400,"""apple""",4,0,0
522373639,"""dometic""",1,0,0


In [84]:
# Looks like these user-ids are distributors (nobody need 400 samsung devices)
df_user_brand_event.head(10)

user_id,brand,view,cart,purchase
i64,str,u32,u32,u32
554090147,"""apple""",37,24,26
548891541,"""ariston""",33,17,19
541337272,"""samsung""",34,24,17
520695559,"""apple""",23,7,16
531974857,"""samsung""",24,11,14
512730829,"""samsung""",22,19,14
543312954,"""samsung""",25,13,13
541311429,"""xiaomi""",25,0,12
524487175,"""redmond""",16,0,12


In [85]:
df_user_brand_event = df_user_brand_event.with_columns(
    (pl.col("purchase") / pl.col("view")).alias("pct_buy_views")
)

In [87]:
df_user_brand_event.head()

user_id,brand,view,cart,purchase,pct_buy_views
i64,str,u32,u32,u32,f64
554090147,"""apple""",37,24,26,0.702703
548891541,"""ariston""",33,17,19,0.575758
541337272,"""samsung""",34,24,17,0.5
520695559,"""apple""",23,7,16,0.695652
531974857,"""samsung""",24,11,14,0.583333


In [88]:
df_user_brand_event = df_user_brand_event.with_columns(
    pl.when(pl.col("pct_buy_views").is_infinite())
      .then(None)
      .otherwise(pl.col("pct_buy_views"))
)

In [90]:
df_user_brand_event.sample(5)

user_id,brand,view,cart,purchase,pct_buy_views,literal
i64,str,u32,u32,u32,f64,f64
526941786,"""brw""",3,0,0,0.0,0.0
531746605,"""redmond""",2,0,0,0.0,0.0
512821524,"""carters""",3,0,1,0.333333,0.333333
551664150,"""elari""",1,0,0,0.0,0.0
515669566,"""acer""",4,0,0,0.0,0.0
